# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f69c53d71c0>
├── 'a' --> tensor([[-0.5861,  0.2685, -1.3736],
│                   [-2.0538,  0.5553,  0.5723]])
└── 'x' --> <FastTreeValue 0x7f69c53d73a0>
    └── 'c' --> tensor([[-1.6836, -0.9774, -0.9544, -0.8893],
                        [-1.4721,  1.3531, -0.2358, -0.4090],
                        [-0.1170,  0.4426,  0.5064,  0.4661]])

In [4]:
t.a

tensor([[-0.5861,  0.2685, -1.3736],
        [-2.0538,  0.5553,  0.5723]])

In [5]:
%timeit t.a

88.8 ns ± 0.115 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f69c53d71c0>
├── 'a' --> tensor([[-0.2431,  0.4169, -0.7225],
│                   [-0.4205, -0.4230, -0.0410]])
└── 'x' --> <FastTreeValue 0x7f69c53d73a0>
    └── 'c' --> tensor([[-1.6836, -0.9774, -0.9544, -0.8893],
                        [-1.4721,  1.3531, -0.2358, -0.4090],
                        [-0.1170,  0.4426,  0.5064,  0.4661]])

In [7]:
%timeit t.a = new_value

90.6 ns ± 0.0868 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.5861,  0.2685, -1.3736],
               [-2.0538,  0.5553,  0.5723]]),
    x: Batch(
           c: tensor([[-1.6836, -0.9774, -0.9544, -0.8893],
                      [-1.4721,  1.3531, -0.2358, -0.4090],
                      [-0.1170,  0.4426,  0.5064,  0.4661]]),
       ),
)

In [10]:
b.a

tensor([[-0.5861,  0.2685, -1.3736],
        [-2.0538,  0.5553,  0.5723]])

In [11]:
%timeit b.a

77.2 ns ± 0.0945 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.2895,  1.3824, -1.2652],
               [-0.7815, -1.1886,  0.1977]]),
    x: Batch(
           c: tensor([[-1.6836, -0.9774, -0.9544, -0.8893],
                      [-1.4721,  1.3531, -0.2358, -0.4090],
                      [-0.1170,  0.4426,  0.5064,  0.4661]]),
       ),
)

In [13]:
%timeit b.a = new_value

645 ns ± 4.56 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.06 µs ± 2.87 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.8 µs ± 17.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

168 µs ± 668 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

168 µs ± 1.05 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f69c5452dc0>
├── 'a' --> tensor([[[-0.5861,  0.2685, -1.3736],
│                    [-2.0538,  0.5553,  0.5723]],
│           
│                   [[-0.5861,  0.2685, -1.3736],
│                    [-2.0538,  0.5553,  0.5723]],
│           
│                   [[-0.5861,  0.2685, -1.3736],
│                    [-2.0538,  0.5553,  0.5723]],
│           
│                   [[-0.5861,  0.2685, -1.3736],
│                    [-2.0538,  0.5553,  0.5723]],
│           
│                   [[-0.5861,  0.2685, -1.3736],
│                    [-2.0538,  0.5553,  0.5723]],
│           
│                   [[-0.5861,  0.2685, -1.3736],
│                    [-2.0538,  0.5553,  0.5723]],
│           
│                   [[-0.5861,  0.2685, -1.3736],
│                    [-2.0538,  0.5553,  0.5723]],
│           
│                   [[-0.5861,  0.2685, -1.3736],
│                    [-2.0538,  0.5553,  0.5723]]])
└── 'x' --> <FastTreeValue 0x7f68f5584ca0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

38.3 µs ± 45.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f69c03c0730>
├── 'a' --> tensor([[-0.5861,  0.2685, -1.3736],
│                   [-2.0538,  0.5553,  0.5723],
│                   [-0.5861,  0.2685, -1.3736],
│                   [-2.0538,  0.5553,  0.5723],
│                   [-0.5861,  0.2685, -1.3736],
│                   [-2.0538,  0.5553,  0.5723],
│                   [-0.5861,  0.2685, -1.3736],
│                   [-2.0538,  0.5553,  0.5723],
│                   [-0.5861,  0.2685, -1.3736],
│                   [-2.0538,  0.5553,  0.5723],
│                   [-0.5861,  0.2685, -1.3736],
│                   [-2.0538,  0.5553,  0.5723],
│                   [-0.5861,  0.2685, -1.3736],
│                   [-2.0538,  0.5553,  0.5723],
│                   [-0.5861,  0.2685, -1.3736],
│                   [-2.0538,  0.5553,  0.5723]])
└── 'x' --> <FastTreeValue 0x7f68fd772ee0>
    └── 'c' --> tensor([[-1.6836, -0.9774, -0.9544, -0.8893],
                        [-1.4721,  1.3531, -0.2358, -0.4090],
                 

In [23]:
%timeit t_cat(trees)

36.1 µs ± 48 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

77.7 µs ± 66.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-1.6836, -0.9774, -0.9544, -0.8893],
                       [-1.4721,  1.3531, -0.2358, -0.4090],
                       [-0.1170,  0.4426,  0.5064,  0.4661]],
              
                      [[-1.6836, -0.9774, -0.9544, -0.8893],
                       [-1.4721,  1.3531, -0.2358, -0.4090],
                       [-0.1170,  0.4426,  0.5064,  0.4661]],
              
                      [[-1.6836, -0.9774, -0.9544, -0.8893],
                       [-1.4721,  1.3531, -0.2358, -0.4090],
                       [-0.1170,  0.4426,  0.5064,  0.4661]],
              
                      [[-1.6836, -0.9774, -0.9544, -0.8893],
                       [-1.4721,  1.3531, -0.2358, -0.4090],
                       [-0.1170,  0.4426,  0.5064,  0.4661]],
              
                      [[-1.6836, -0.9774, -0.9544, -0.8893],
                       [-1.4721,  1.3531, -0.2358, -0.4090],
                       [-0.1170,  0.4426,  0.5064,  0.4661]],

In [26]:
%timeit Batch.stack(batches)

96.8 µs ± 101 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-1.6836, -0.9774, -0.9544, -0.8893],
                      [-1.4721,  1.3531, -0.2358, -0.4090],
                      [-0.1170,  0.4426,  0.5064,  0.4661],
                      [-1.6836, -0.9774, -0.9544, -0.8893],
                      [-1.4721,  1.3531, -0.2358, -0.4090],
                      [-0.1170,  0.4426,  0.5064,  0.4661],
                      [-1.6836, -0.9774, -0.9544, -0.8893],
                      [-1.4721,  1.3531, -0.2358, -0.4090],
                      [-0.1170,  0.4426,  0.5064,  0.4661],
                      [-1.6836, -0.9774, -0.9544, -0.8893],
                      [-1.4721,  1.3531, -0.2358, -0.4090],
                      [-0.1170,  0.4426,  0.5064,  0.4661],
                      [-1.6836, -0.9774, -0.9544, -0.8893],
                      [-1.4721,  1.3531, -0.2358, -0.4090],
                      [-0.1170,  0.4426,  0.5064,  0.4661],
                      [-1.6836, -0.9774, -0.9544, -0.8893],
                   

In [28]:
%timeit Batch.cat(batches)

180 µs ± 518 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

400 µs ± 5.46 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
